# JPEG Analysis
## An in-depth look at the JPEG File Interchange Format with Python3
#### Skyler Rankin 2019
##### *Version 0.1*

### Introduction
This notebook attempts to explore the mechanisms at play behind one of the most widely used image file formats. In short, by parsing the file byte by byte, and using a specification reference, we can interpret all of the information stored in the file, and even generate the corresponding image without a pre-built image viewer. The primary reference used will be the report given by Ecma-Internation found [here](https://www.ecma-international.org/publications/files/ECMA-TR/ECMA%20TR-098.pdf).

### Table of Contents
#### [Section 1](#1) | Importing an Image
#### [Section 2](#2) | Basic JPEG Structure
#### [Section 3](#3) | Parsing Segments: Part 1
#### [Section 4](#4) | Huffman Encoding


### Section 1: Importing an Image<a name="1"></a>
First, we must select an image to work with. I have included a small default image with this repository for testing, but feel free to add your own by adding the file to the project root and changing the filename in the code below.

In [1]:
from IPython.display import Image
image_filename = "colors.jpg"
Image(image_filename, width=100)

### Section 2: Basic JPEG Structure<a name="2"></a>
The structure of a JPEG file relies upon a system of **markers**. These are specific 2-byte sequences that are reserved for a specific purpose, so that when you see one in the file, you can be sure of its meaning. For convenience, I will represent these byte sequences as hex values preceeded by `0x`, meaning the decimal number `78` is represented as `1001110` as binary in the image file, but is written as `0x4E` in hex for our purposes. In the JPEG File Format, all markers are preceeded by `0xFF`, which makes the marker two bytes in total.
The first markers to look for are the **SOI** and **EOI** markers. These signify **start of image** and **end of image** respectively.

Marker | Value
--- | --- 
SOI | 0xFFD8
EOI | 0xFFD9

We can be sure that no important information for the image will come before the `SOI` marker, and no information will come after the `EOI`. The information between these markers can be referred to as a segment. A common idea for structuring segments is using a **header**. A header is a small sequence of bytes at the beginning of a segment that describe what that segment contains. The specifics for the headers are dependent on the marker type, but we will see many of these headers as we parse the image file. An example of this is the `SOF0` marker, which denotes the **start of frame 0**. This segment uses the marker `0xC0` and contains information regarding the image data. The specifics of extracting information from headers will come in the next section.

Before we can parse these headers and segments, it helps to define a function to convert the bytes read from the file as Python `byte` objects into pure integers. This makes storing and manipulating their values simpler later on. The following function `hti`, "Hex To Integer", does just that: takes in a byte and returns the integer representation.

In [6]:
def hti(h):
    return int.from_bytes(h, byteorder='big')

print("Hex value 0xD8 converted with function: ",hti(b'\xD8'))
print("Actual integer value: ",16*13+8)

Hex value 0xD8 converted with function:  216
Actual integer value:  216


### Section 3: Parsing Segments Part 1<a name="3"></a>

The general structure I will use to parse segments is defining a function for each of the segments we want to parse. These functions will take in the file we are reading as the argument `file`. An important function that will be repeatedly used is `read`. This function takes in an integer representing the number of bytes to read and return from the given file object. So when we type `file.read(2)`, we read the next two bytes from the file and return that 2 byte value. Afterwards, I will always use my function previously defined, `hti`, to convert those bytes into an integer.

For all segments, the data of that segment starts immediately after the segment marker. So as soon as our program reads `0xFFC0`, the marker for SOFO, we can call the function to parse SOFO, and any subsequent calls to read will be reading the bytes that are next in the file.

##### Start of Frame Parsing

As mentioned, the first segment we will write code to parse is the **start of frame** segment. According to the documentation, the first 2 bytes of a SOFO segment denote the length of the segment. That's easy enough to handle: we store that value into a variable called `length`, as seen on line `3`. After that come the precision, height, width, and number of components. We store all of those in the same fashion.

While pieces of information like width and height are simple to read, since they are of fixed length, other data is a little trickier. The first variable length information we will see is the `components` data within the SOFO segment. This is a list of image components, each containing 3 bytes:
1. ID - Which part of the YCrCb color space this component holds
2. Sampling Factor - This is used later on
3. Quantization Table - The ID of the quantization table to be used

This piece of information is variable length because there could be 1 component, or there could be 3, or some other value: we don't know for every image how many there will be. As such, we cannot hard-code calls to `read`. Instead, we will use the provided `length` value. Consider **Figure 3.1** below. The first yellow block denotes the 2 byte marker `0xFFC0`. All green blocks represent some piece of information we will read. The light green block is the block of variable length.
<img src="images/4.1.jpg">
The first value in the segment is the `length`, which is a 2 byte value, so it makes up bytes 3 and 4. Let's assume that after reading those 2 bytes, we saw `length=17`. Following the documentation, byte 5 will be the data precision, bytes 6 and 7 are the height, 8 and 9 are the width, and byte 10 is the number of components. Let's say the number of components read was `3`.

Since each component contains 3 bytes, we know that there will be `9` total bytes we need to read to get all 3 components. Furthermore, since the first 5 pieces of information took 8 bytes and the components took 9 bytes, that makes 17 bytes total. This matches the `length` value exactly.

With that understanding out of the way, we can look at the function itself. As mentioned, we take in a reference to the file. The function then reads all of the fixed length variables and stores them. It loops over the the number of components, adding them to an array. Finally, it returns a dictionary that maps the names of the values to the data we got from that segment.

In [3]:
def parse_SOF0(file):
    #length = 8 + (#components * 3)
    length = hti(file.read(2))
    #bits/sample
    precision = hti(file.read(1))
    height = hti(file.read(2))
    width = hti(file.read(2))
    #1 = grey scale, 3 = YCbCr, 4 = CMYK
    num_components = hti(file.read(1))
    components = []
    for i in range(int((length - 8) / 3)):
        #1 = Y, 2 = Cb, 3 = Cr, 4 = I, 5 = Q
        component_id = hti(file.read(1))
        sampling_factor = hti(file.read(1))
        q_table = hti(file.read(1))
        components.append({
            "id": component_id,
            "sampling_factor": sampling_factor,
            "q_table": q_table
        })
    return {
        "length": length,
        "precision": precision,
        "height": height,
        "width": width,
        "num_components": num_components,
        "components": components
    }

Notice that there is some further information we can extract from these values. Since the number of components denotes how many color spaces we need to display the image, it is clear that if there is only 1, we are parsing a purely gray-scale image. If there are 3 components, then we are dealing with the common YCbCr system: more on this later.

##### Define Quantization Table Parsing

The next segment we will tackle is the **DQT** segment. This segment contains the values for a quantization table which will later be used for decoding image data. These tables are basically 8x8 matrices of integers. As with the SOFO segment, the first piece of information is a length value, which is again 2 bytes. After the length, there is a single information byte for the quantization table. Last is the table itself, which contains `64` values if the precision is 0 (8 bit) and `128` values if the precision is 1 (16 bit). What's tricky here is that a segment may contain multiple quantization tables, each with a single information bit and a series of values. Thus, the only way to tell how many tables you have is to use the `length` value while parsing. Consider **Figure 4.2** below.
<img src="images/4.2.jpg">
The yellow block denotes the segment marker `0xFFDB`. After that, we can see the 2 byte length value, bytes 3 and 4. Let's assume that this value was 132.
After the length, we enter a light green block, which contains a quantization table. We first read byte 5, the information byte of a table. Afterwards, we read the 64 values for that table, bytes 6 through 69.
At this point, from the tables alone we would not know if we had completed the segment. But since the total length was 132, and we had read 2 bytes for the length itself plus 65 bytes for the first table, there are `132 - 2 - 65 > 0` some number of bytes remaining. So we continue, assuming that there will be at least 1 more complete table to read. Again, reading 1 information bit, byte 70, and 64 values, bytes 71 through 134, giving us our second quantization table from this segment. At this point, `132` bytes have been read, so the segment is completed.

In [4]:
import numpy as np

def parse_DQT(file):
    #length = 6+2(number of components)
    length = hti(file.read(2))
    tables = []
    pos = length-2
    
    while pos > 0:
        qt = hti(file.read(1))
        number = qt&0xF
        precision = qt >> 4
        table = {
            "id": number,
            "precision": precision
        }
        data = []
        for i in range(64*(precision+1)):
            data.append(hti(file.read(1)))
        matrix = np.array(data)
        matrix = matrix.reshape(8,8)
        
        table["matrix"] = matrix
        tables.append(table)
        
        pos-=(1 + (64*(1+precision)))
        
    return {
        "length": length,
        "tables": tables
    }

### Section 4: Huffman Encoding<a name="4"></a>

Image data isn't stored as raw pixel values, but as huffman encoded symbols.

In [5]:
def generate_bitstrings(table):
    bitstrings = {}    
    value = 0
    for i in range(1,table["last non-zero"]+2):
        if table["lengths"][i-1] == 0:
            continue
        strings = []
        for code in range(table["lengths"][i-1]):
            strings.append(value)
            value+=1
        bitstrings[i] = strings
        value <<= 1
    return bitstrings

In [ ]:
class TreeNode:
    def __init__(self, value=None, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right
    
    def get_node(self, path):
        if path == '':
            return self
        return self.left.get_node(path[1:]) if path[0:1] == '0' else self.right.get_node(path[1:])
    
def make_node(root, path, value):
    if path=="":
        root.value = value
    elif path[:1] == "0":
        if root.left == None:
            root.left = TreeNode()
        make_node(root.left, path[1:], value)
    else:
        if root.right == None:
            root.right = TreeNode()
        make_node(root.right, path[1:], value)
        
def generate_tree(table):
    bitstrings = generate_bitstrings(table)
    root = TreeNode(value=None)
    value_index = 0
    for level in range(1, table["last non-zero"]+2):
        if level not in bitstrings.keys():
            continue
        for s in bitstrings[level]:
            binary = str(bin(s))[2:]
            path = ("0"*(level-len(binary)))+binary
            make_node(root, path, table["symbols"][value_index])
            value_index+=1
    return root

#import graphviz
#graph = graphviz.Digraph()
#graph.edge("a", "b")
#graph.edge("a", "c")
#graph.edge("b", "d")
#graph.edge("b", "e")
#graph.edge("c", "t")
#graph

In [ ]:
def test_huffman_tree(table, tree):
    bitstrings = generate_bitstrings(table)
    print("---Testing Huffman Tree Implementation")
    print("bitstrings: {}".format(bitstrings))
    code = 0
    for level in bitstrings:
        print("Level = {}".format(level))
        for string in bitstrings[level]:
            binary = str(bin(string))[2:]
            path = ("0"*(level-len(binary)))+binary
            symbol = tree.get_node(path).value
            print("\t{}: Expected {}, Found {}".format(path, table["symbols"][code], symbol))
            code+=1
    

format is Y Cb Cr for each MCU, in order. 

In [ ]:
bit_buffer = ""
def read_bit(file):
    global bit_buffer
    if bit_buffer == "":
        byte = file.read(1)
        bit_buffer = str(bin(hti(byte)))[2:]
        if len(bit_buffer) < 8:
            bit_buffer = "0"*(8 - len(bit_buffer)) + bit_buffer
        #print("\tbit_buffer update: {}".format(bit_buffer))
    bit = int(bit_buffer[:1])
    bit_buffer = bit_buffer[1:]
    return bit

In [ ]:
def parse_DHT(file):
    length = hti(file.read(2))
    pos = length
    count = 0
    tables = []

    while pos > 0 and count < 4:
        count+=1
        ht_info = hti(file.read(1))
        pos-=1

        number = ht_info&0xF
        ht_type = ht_info >> 4
        
        table = {"id": number, "type": ht_type}
        
        lengths = []
        last = None
        for i in range(16):
            l = hti(file.read(1))
            lengths.append(l)
            if l != 0:
                last = i
        table["lengths"] = lengths
        table["last non-zero"] = last
        pos-=16
                
        symbols = []
        for i in range(16):
            if lengths[i] > 0:
                for j in range(lengths[i]):
                    symbols.append(hti(file.read(1)))
                    pos-=1
        table["symbols"] = symbols        
        tables.append(table)
        
    trees = {}
    for table in tables:
        if table["id"] not in trees.keys():
            trees[table["id"]] = {}
            
        if table["type"] == 0:
            trees[table["id"]]["DC"] = generate_tree(table)
            #test_huffman_tree(table, trees[table["id"]]["DC"])
        elif table["type"] == 1:
            trees[table["id"]]["AC"] = generate_tree(table)
        
    return {
        "length": length,
        "trees": trees
    }

In [ ]:
def parse_SOS(file):
    #length = 6+2*(number of components)
    length = hti(file.read(2))
    number_components = hti(file.read(1))
    components = []
    for i in range(number_components):
        cid = hti(file.read(1))
        table = hti(file.read(1))
        components.append({
            "id": cid,
            "AC": table&0xF,
            "DC": table>>4
        })
    file.read(3)
    
    return {
        "length": length,
        "number of components": number_components,
        "components": components
    }    

In [ ]:
def read_image_data(file):
    mcus = []
    mcu = []
    for i in range(data["SOS"][0]["number of components"]):
        trees = data["DHT"][0]["trees"]
        component = data["SOS"][0]["components"][i]
        #print("Reading Component {}: {}".format(i, component))
        
        #DC value
        tree = trees[component["DC"]]["DC"]
        while tree.value == None:
            bit = read_bit(file)
            tree = tree.get_node(str(bit))
        dct = [tree.value]
        #print("DC Value = {}".format(tree.value))
        
        #63 AC values
        tree = trees[component["AC"]]["AC"]
        for i in range(63):
            while tree.value == None:
                bit = read_bit(file)
                tree = tree.get_node(str(bit))
            dct.append(tree.value)
            if tree.value == 0:
                break;
            tree = trees[component["AC"]]["AC"]
        
        if len(dct) < 64:
            dct = dct + (64-len(dct))*[0]
        dct = np.array(dct)
        dct = dct.reshape(8,8)
        mcu.append(dct)
    mcus.append(mcu)
    print("MCUs")
    print(mcus)

In [ ]:
def next_segment(file):
    markers = {
        0xFFD8: "SOI",
        0xFFE0: "APP0",
        0xFFC0: "SOF0",
        0xFFC4: "DHT",
        0xFFDB: "DQT",
        0xFFDA: "SOS",
        0xFFD9: "EOI",
    }
    
    b1 = file.read(1)
    b2 = file.read(1)

    while hti(b1+b2) not in markers.keys():
        b1 = b2
        b2 = file.read(1)
        
    m = hti(b1+b2)
    return markers[m]

file = open(image_filename, "rb")
data = {
    "SOF0": [],
    "DHT": [],
    "DQT": [],
    "SOS": []
}

for i in range(10):
    segment = next_segment(file)
    if segment == "SOF0":
        data[segment].append(parse_SOF0(file))
    elif segment == "DHT":
        data[segment].append(parse_DHT(file))
    elif segment == "DQT":
        data[segment].append(parse_DQT(file))
    elif segment == "SOS":
        data[segment].append(parse_SOS(file))
        read_image_data(file)
        break
file.close()

cosine

In [ ]:
from scipy.fftpack import idctn
q1 = data["DQT"][0]["tables"][0]["matrix"]
q2 = data["DQT"][0]["tables"][1]["matrix"]

a = np.array([[ 5,  2, 33,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0]])

#add in difference from previous DC value

#entry-for-entry product with quantization matrix
a = np.multiply(a, q2)

#inverse DCT
a = idctn(a, norm="ortho")

#round all values
a = np.round(a)

#add 128 to all entries
img = a + 128

print(img)
#clip values outside [0, 255]

test = np.array([[-416, -33, -60, 32, 48, -40, 0, 0],
                [0, -24, -56, 19, 26, 0, 0, 0],
                [-42, 13, 80, -24, -40, 0, 0, 0],
                [-42, 17, 44, -29, 0, 0, 0, 0],
                [18, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0]])
#r = idctn(test, norm="ortho")
#r = np.round(r)

In [ ]:
import matplotlib.pyplot as plt
img = [[[45,255,43]]]
plt.imshow(img)
plt